# Notas Introductorias

## 1) Sobre el proceso de deconvolución y el rol del algoritmo de CLEAN en síntesis de imagenes en radioastronomía

> ### Conceptos Clave:
>> - Fast Fourier Transform: Es más rapido y simple. O(NlogN) para una imagen $2^N x 2^N$
>> - El plano $(u,v)$ es la transformada de Fourier del cielo.
>> - $V(u,v)$: Función de visibilidad compleja (complex visibility function), es la transformada de Fourier en 2D de  la distribución de brillo del cielo $T(l,m)$ (sky brightness distribution). <br>
Matemáticamente, se expresan como: <br>
$$V(u,x) = \iint T(l,m) e^{-i2\pi(ul+vm)} \,dl \,dm$$
$$T(l,m) = \iint V(u,v) e^{i2\pi(ul+vm)} \,du ,dv$$
Donde:
    - l,m son los angulos, respectivamente, Este-Oeste y Norte-Sur en el plano tangente. [rad]
    - u,v son las frecuencias espaciales, respectivamente, Este-Oeste y Norte-Sur. [wavelengths]
>> - $T(l,m)$: brightness distribution of the sky.
>> - $S(l,m)$: True Sampling Distribution Function.
>> - $s(l,m)$: transformada de Fourier de $S(l,m)$, point spread function, tambien llamado "Dirty Beam".
>> - La Dirty Image se forma a partir de la convolución del dirty beam y la distribución de brillo del cielo.





> ### Visibility and sky Brightness
>> Gracias a la transformada de Fourier, podemos hacer la relación:
    $$V(u,v) \rightarrow T(l,m)$$
    
>> La idea básica es que con muestreos de  la visibilidad $V(u,v)$ para suficientes puntos (u,v) usando antenas de pequeña abertura, distribuidas, se puede sintentizar una antena de larga apertura, de tamaño $(u_{max},v_{max})$. Para llenar el plano $(u,v)$ se utiliza la rotación de la tierra, además se recofigura la posición física de las antenas para mayor cantidad de muestras. <br>
<img src = "DirtyBeamDirtyImage.png">
**Imagen 1:** Proceso de la medición, desde la función de muestreo $S(u,v)$ hasta la Dirty Image $T^{D}(l,m)$. Créditos a David J. Wilner, "Lecture II: Imaging and Deconvolution in Radio Interferometry", NRAO.<br>

>> Podemos recuperar una imagen desde las muestras incompletas y ruidosas de su transformada de Fourier para el análisis:
- Calcular la Transformada de Fourier de $V(u,v)$ para obtener $T^{D}(l,m)$.
- Es dificil hacer ciencia con la dirty image, necesitamos una imagen limpia ("Cleaned").
- Deconvolucionar $s(l,m)$ desde $T^{D}(l,m)$ para determinar un modelo de T(l,m), esto es lo que hace CLEAN, es un algoritmo deconvolutivo.
- Trabajamos con el modelo de $T(l,m)$.

>> hay que hacer un "gridding" usado para remuestrear $V(u,v)$ para poder usar FFT (Fast Fourier Transform).

> ### Deconvolución
>> - La deconvolución usa tecnicas no lineales para interpolar/extrapolar muestras de V(u,v) hacia regiones del plano $(u,v)$ no muestreadas.
>> - Su objetivo es encontrar un modelo de $T(l,m)$ compatible con los datos.
>> - Requiere supuestos previos sobre $T(l,m)$ para elegir distribuciones "invisibles" plausibles, para llenar las partes del plano de Fourier no medidas.

>> **Algoritmos de deconvolución**:
>>> - **CLEAN**: Algoritmo dominante en radioastronomía.
    >>>> - Supuesto previo: $T(l,m)$ es una colección de fuentes puntuales.
    >>>> - Ajusta y sustrae el "synthesized beam" **iterativamente**.
    >>>> - Existen variantes desarrolladas para mayor eficiencia computacional, model visibility substraction, para tratar de mejor forma con estructuras de emission extendidas, etc.

>>> - **Maximum Entropy Method(MEM)**: Alternativa a CLEAN, no es tan popular.
    >>>> - Supuesto previo: $T(l,m)$ es suave y positiva.
    >>>> - Define "*suavidad*" a través de una expresión matemática para entropia ( Gull and Skilling 1983), encuentra la imagen más suave consistente con los datos.
    >>>> - Al parecer si posee fundamento matemático suficiente como para poder generar una medida de error cuantitativo.
    >>>> - Su principal desventaja es que suele ser más lento que CLEAN
    >>>> - Principal ventaja: se desempeña mejor que clean para imagenes con mas de un millon de pixeles (https://archive.lib.msu.edu/crcmath/math/math/m/m118.htm).
    >>>> - Su resolución depende del ratio señal/ruido, el cual se le debe especificar, por lo que la resolución depende de la imagen y varia a lo largo del mapa.
    >>>> - Tiene base estadística a diferencia de CLEAN.
    >>>> - Puede llegar a ser mas rapido que clean bajo ciertas condiciones.
    
## 2) El algoritmo de CLEAN
> - Es un algoritmo no lineal.
> - No se puede utilizar algoritmos de deconvolucion lineal como Filtrado de Wiener o Filtrado Inverso, porque no se pueden aplicar a problemas con distribuciones invisibles. (Por ejemplo, muestreo incompleto del plano de frecuencias espaciales).
> - Ideado originalmente para fuentes puntuales, pero **se ha visto que funciona bien para fuentes extendidas cuando se le da un modelo inicial razonable**.
$$b' * I = I'$$
Donde $b'$ es el Dirty Beam, $I'$ es el Dirty map, ambos en el plano $(u,v)$  y  $f * g$ denota una convolución.
> El algoritmo comienza con una aproximación inicial $I_0 = 0$, a la n-ésima iteración, busca por el mayor valor en el mapa residual:
$$I_n = I'-b' * I_{n-1}$$



>>### Procedimiento Básico:
>>> Inicializar:
     - Un mapa residual para el dirty map.
     - Una Clean Component list.

>>> 1) Identificar el peak más alto en el mapa residual como una fuente puntual.<br>
>>> 2) Substraer una fracción de este peak del mapa residual usando un dirty beam escalado: $s(l,m) x gain$.<br>
>>> 3) Añadir la localización y amplitud de esta fuente puntual a la lista *Clean Component*.<br>
>>> 4) Volber al paso 1), a menos que se haya alcanzado un criterio de detención.<br>

>>> **Criterios de detención:**
>>> - Max(residual map) < threshold = multiple rms (si el ruido es limitado).
>>> - Max(residual map) < threshold = fraction of dirty map maximum (Si el rango dinámico es limitado).
>>> - Maximo numero de componentes en la Clean Components alcanzado.

>>> **Loop gain:**
>>> - Buenos resultados para g=0.1 a 0.3.
>>> - Valores bajos funcionan mejor para emisiones mas suaves, g = 0.05.

>>> Al parecer es fácil incluir información *a priori* sobre donde buscar en el Dirty Map para buscar por *Clean Components*, usando "boxes" o "masks" <font color = "red">(?)</font>, aunque es riesgoso porque se puede estar obviando zonas de interés.

>>> **Ultimo Paso: Crear la imagen restaurada:**

>>> 1) Hacer una imagen modelo con todos las fuentes puntuales de la lista *Clean Components*.<br>
>>> 2) Convolucionar las fuentes puntuales con una Gausiana Elíptica, ajustar al lóbulo central <font color = "red">(?)</font> del Dirty Beam ("Clean Beam").<br>
>>> 3) La imagen resultante es un estimador del verdadero brillo del cielo (sky brightness) $T(l,m)$.<br>
>>> 4) Unidades de la imagen restaurada son principalmente JY por área del Clean Beam = Intensity (brightness temperature).<br>

>>> **La imagen restaurada no se ajusta en realidad a las visibilidades observadas** <font color = "red">(?)</font>

> - Al finalizar la substracción de componentes, se asume que la distribución residual de brillo consiste principalmente de ruido.
> - Para disminuir atributos de alta frecuencia espacial que puedan ser incorrectamente extrapolados desde los datos medidos, cada componente de CLEAN es convolucionado con el llamado *CLEAN Beam* $b$, el cual es simplemente una version suavizada de la funcion de muestreo (Dirty Beam). Por lo general se usa una Gausiana.
> **EL AIPS de la NRAO tiene algoritmos de deconvolución 2D en las tareas DCONV y UVMAP**.<br>

## 3) Optimizando CLEAN

### (a) Clark Variation
> - 2-10 veces más rapido que implementación clásica de CLEAM (Hogbom).
> - Implementado en la rutina APCLN de AIPS.
> - Es el algoritmo por defecto implementado por la función *imager.clean* de CASA.
> - La limpieza se divide en dos tipos de ciclos, mayor y menor
>> - En los ciclos menores, solo los puntos más luminosos son limpiados, usando un subset de funciones de propagación (spread functions).
>> - En los ciclos mayores, los putos encontrados en los ciclos menores son substraidos correctamente usando una convolucion basada en FFT.

### (b) Cornwell Smoothness Stabilized Variation
> - Se desarrolló porque, cuando se trata con estructuras extendidas 2-dimensionales, CLEAN puede producir artefactos en la forma de lineas de bajo nivel- alta frecuencia a lo largo de la estructura más luminosa.
> - Implementado en la rutina APCLN y MX de AIPS
       
      


# Análisis Papers Pablo Román

## Hermite-Gaussian functions for image synthesis
> - Interderómetros realizan un muestreo irregular de la transformada de Fourier del cielo llamadas "visibilidades" (visibilities).
> - El problema de resolución inversa se le denomina "Sintesis de Imagenes".
> - El paper propone usar una representacion no-local del problema inverso (sintesis de imagenes), lo propuesto esta basado en el uso de funciones Hermite-Gaussian(HG), las cuales son un set completo de vectores propios para el operador de Fourier que se usa para representar el problema
> - El problema de sintetizar imagenes corresponde a estimar la mejor aproximación de imagen desde un muestreo de la su transformada de fourier escaso y disperso.
> - Asumen que el modelo del campo de visibilidad tiene un ruido correlacionado con una delta dirac (Ruido blanco gaussiano por ejemplo).
> - El problema es tener una buena estimación del valor esperado $E(I(Z))$ y varianza del campo aleatorio (random field) acorde a las visibilidades observadas $\{V_{k}^{O}\}_{k=1}^{N}$ y desviación $\{\sigma_{k}^{O}\}_{k=1}^{N}$.
> - Los metodos actuales de sintesis de imagenes solo estan realcionados con $E(I)$ y los requerimientos de varianza son interpretados desde los archivos residuales.
> - El problema de la sintesis de imagenes es que las soluciones posibles no son unicas y no dependen continuamente de los datos de entrada.
> - Distinguen dos aproximaciones principales para resolver el problema de la sintesis de imagenes e radioastronomia:<br>
>> 1) Heurísiticas de tipo greedy consistentes en algoritmos que acumulen componentes de funciones para construir la imagen.
>> 2) 